<a href="https://colab.research.google.com/github/capajaro/CSS_UN/blob/main/CSS_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>